In [1]:
import random
import sys
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLUT.fonts import GLUT_BITMAP_HELVETICA_12

# Global variables
WIDTH, HEIGHT = 1200, 800  # Increased window size
array = []
n = 50
min_val = 0
max_val = 100
sorting = False
ascending = True
current_algo = "Bubble Sort"
color_list = []

# Time complexities of sorting algorithms
algorithm_complexities = {
    "Bubble Sort": "O(n^2)",
    "Insertion Sort": "O(n^2)",
    "Selection Sort": "O(n^2)",
    "Merge Sort": "O(n log n)",
    "Quick Sort": "O(n log n)"
}

# Function to draw text on OpenGL window
def draw_text(value, x, y):
    glColor3f(0, 0, 0)
    glRasterPos2f(x, y)
    for char in value:
        glutBitmapCharacter(GLUT_BITMAP_HELVETICA_12, ord(char))

# Function to draw algorithm info
def draw_algorithm_info():
    global current_algo
    glLoadIdentity()
    glColor3f(0, 0, 0)
    draw_text(f"Current Algorithm: {current_algo}", -0.95, 0.85)
    draw_text(f"Time Complexity: {algorithm_complexities[current_algo]}", -0.95, 0.80)

# Function to draw instructions
def draw_instructions():
    instructions = [
        "Press SPACE to Start/Stop Sorting",
        "Press 'r' to Reset Array       Press 'a' for Ascending Order      Press 'd' for Descending Order",
        "Press '1' for Bubble Sort    Press '2' for Insertion Sort    Press '3' for Selection Sort      Press '4' for Merge Sort    Press '5' for Quick Sort"
    ]
    glColor3f(0, 1, 0)
    for i, instruction in enumerate(instructions):
        draw_text(instruction, -0.95, 0.75 - i * 0.05)  # Adjusted position to the top

# Display function for OpenGL
def display():
    glClear(GL_COLOR_BUFFER_BIT)
    draw_algorithm_info()
    draw_instructions()
    draw_array()
    glutSwapBuffers()

# Generator functions for sorting algorithms
def bubble_sort():
    global array
    for i in range(len(array) - 1):
        for j in range(len(array) - 1 - i):
            if (array[j] > array[j + 1] and ascending) or (array[j] < array[j + 1] and not ascending):
                array[j], array[j + 1] = array[j + 1], array[j]
                yield True

def insertion_sort():
    global array
    for i in range(1, len(array)):
        key = array[i]
        j = i - 1
        while j >= 0 and ((key < array[j] and ascending) or (key > array[j] and not ascending)):
            array[j + 1] = array[j]
            j -= 1
            yield True
        array[j + 1] = key
        yield True

def merge_sort():
    global array
    yield from merge_sort_recursive(array, 0, len(array) - 1)

def merge_sort_recursive(arr, left, right):
    if left >= right:
        return
    middle = (left + right) // 2
    yield from merge_sort_recursive(arr, left, middle)
    yield from merge_sort_recursive(arr, middle + 1, right)
    yield from merge(arr, left, middle, right)

def merge(arr, left, middle, right):
    left_copy = arr[left:middle + 1]
    right_copy = arr[middle + 1:right + 1]
    left_idx, right_idx = 0, 0
    for k in range(left, right + 1):
        if left_idx < len(left_copy) and (right_idx >= len(right_copy) or 
           (left_copy[left_idx] <= right_copy[right_idx] and ascending) or 
           (left_copy[left_idx] >= right_copy[right_idx] and not ascending)):
            arr[k] = left_copy[left_idx]
            left_idx += 1
        else:
            arr[k] = right_copy[right_idx]
            right_idx += 1
        yield True

def selection_sort():
    global array
    for i in range(len(array)):
        min_idx = i
        for j in range(i + 1, len(array)):
            if (array[j] < array[min_idx] and ascending) or (array[j] > array[min_idx] and not ascending):
                min_idx = j
        if min_idx != i:
            array[i], array[min_idx] = array[min_idx], array[i]
            yield True

def partition(arr, low, high):
    pivot = arr[high]
    i = low - 1
    for j in range(low, high):
        if (arr[j] < pivot and ascending) or (arr[j] > pivot and not ascending):
            i += 1
            arr[i], arr[j] = arr[j], arr[i]
            yield True
    arr[i + 1], arr[high] = arr[high], arr[i + 1]
    yield True
    return i + 1

def quick_sort_recursive(arr, low, high):
    if low < high:
        partition_idx = yield from partition(arr, low, high)
        yield from quick_sort_recursive(arr, low, partition_idx - 1)
        yield from quick_sort_recursive(arr, partition_idx + 1, high)

def quick_sort():
    global array
    yield from quick_sort_recursive(array, 0, len(array) - 1)

# OpenGL functions
def generate_array():
    global array, color_list
    array = [random.randint(min_val, max_val) for _ in range(n)]
    color_list = [(random.random(), random.random(), random.random()) for _ in range(n)]

def draw_array():
    glLoadIdentity()
    for i in range(n):
        x = (i * 2 / n) - 1
        y = -0.7 + (1.4 * array[i] / max_val)  # Adjusted height to leave more space for instructions
        color = color_list[i]
        glColor3f(*color)
        glBegin(GL_QUADS)
        glVertex2f(x, -0.7)
        glVertex2f(x + (2 / n), -0.7)
        glVertex2f(x + (2 / n), y)
        glVertex2f(x, y)
        glEnd()

        # Draw bar values
        draw_text(f"{array[i]}", x + (1 / n), y + 0.02)

def timer(value):
    global sorting
    if sorting:
        if current_algo == "Bubble Sort":
            try:
                next(bubble_sort_generator)
            except StopIteration:
                sorting = False
        elif current_algo == "Insertion Sort":
            try:
                next(insertion_sort_generator)
            except StopIteration:
                sorting = False
        elif current_algo == "Merge Sort":
            try:
                next(merge_sort_generator)
            except StopIteration:
                sorting = False
        elif current_algo == "Selection Sort":
            try:
                next(selection_sort_generator)
            except StopIteration:
                sorting = False
        elif current_algo == "Quick Sort":
            try:
                next(quick_sort_generator)
            except StopIteration:
                sorting = False
    glutPostRedisplay()
    glutTimerFunc(50, timer, 0)

def keyboard(key, x, y):
    global sorting, ascending, current_algo, bubble_sort_generator, insertion_sort_generator, merge_sort_generator, selection_sort_generator, quick_sort_generator
    if key == b' ':
        sorting = not sorting
    elif key == b'r':
        generate_array()
    elif key == b'a':
        ascending = True
    elif key == b'd':
        ascending = False
    elif key == b'1':
        current_algo = "Bubble Sort"
        bubble_sort_generator = bubble_sort()
    elif key == b'2':
        current_algo = "Insertion Sort"
        insertion_sort_generator = insertion_sort()
    elif key == b'3':
        current_algo = "Selection Sort"
        selection_sort_generator = selection_sort()
    elif key == b'4':
        current_algo = "Merge Sort"
        merge_sort_generator = merge_sort()
    elif key == b'5':
        current_algo = "Quick Sort"
        quick_sort_generator = quick_sort()
    glutPostRedisplay()

def main():
    global bubble_sort_generator, insertion_sort_generator, merge_sort_generator, selection_sort_generator, quick_sort_generator
    generate_array()
    bubble_sort_generator = bubble_sort()
    insertion_sort_generator = insertion_sort()
    merge_sort_generator = merge_sort()
    selection_sort_generator = selection_sort()
    quick_sort_generator = quick_sort()
    glutInit(sys.argv)
    glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB)
    glutInitWindowSize(WIDTH, HEIGHT)
    glutCreateWindow(b"Sorting Algorithm Visualizer")
    glClearColor(1, 1, 1, 1)
    glutDisplayFunc(display)
    glutKeyboardFunc(keyboard)
    glutTimerFunc(50, timer, 0)
    glutMainLoop()
    
if __name__ == "__main__":
    main()


: 